<a href="https://colab.research.google.com/github/GmoWilliams/Roomba-MultiAgents/blob/main/Actividad1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

###**Multiagentes de Limpieza Roombas**
#### *Elaborado por:* 
#### - Jesús Enríquez Jaime
#### - Valter Kuhne
#### - José Emilio Flores Figueroa
#### - Guillermo Williams

####*Primero instalamos la herramienta de agentpy*

In [ ]:
!pip install agentpy

####*Importamos las dependencias necesarias para las operaciones*

In [ ]:
# Model design
import agentpy as ap

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns
import IPython
from random import randrange, uniform

####*Definimos la clase para el agente de nuestra categoría: Robot (Los Roombas)*
#####**Setup**
#####Este objeto se moverá en espacios tipo "grid" de la biblioteca agentpy
#####Le asignamos el color 0 (gris)
#####**Find New Cell**
#####Definimos su función para moverse únicamente un espacio en un eje de ocho direcciones

In [ ]:
class Robot(ap.Agent):

    def setup(self):
        """ Initiate agent attributes. """
        self.grid = self.model.grid
        self.random = self.model.random
        self.typeColor = 0

    def find_new_cell(self):
        """ Move to random free spot and update free spots. """
        num1 = 0
        num2 = 0
        while (num1 == 0 and num2 == 0 ):
          num1 = randrange(-1, 2)
          num2 = randrange(-1, 2)
        self.grid.move_by(self, (num1, num2))

####*Definimos la clase para el agente de nuestra segunda categoría: Cell (Las Celdas Sucias)*
#####**Setup**
#####Se inicializan sus valores del mismo modo pero agregando un parámetro para verificar si la celda se ha limpiado

In [ ]:
class Cell(ap.Agent):

    def setup(self):
        """ Initiate agent attributes. """
        self.grid = self.model.grid
        self.random = self.model.random
        self.clean = False
        self.typeColor = 1

####*Definimos la clase CleaningModel donde se ejecutara toda la secuencia del modelo*
#####**Parámetros**
#####Tomamos los valores iniciales dados en los parámetros, calculamos la densidad en la cuadrícula e inicializamos una variable para contabilizar los movimientos de los agentes
#####**Creación de los Agentes y la Cuadrícula**
#####Se inicializan los agentes
#####**Update**
#####Se realiza una iteración para encontrar coincidencias en las coordenadas de ambas categorías de agentes (Roomba y Piso Sucio) en el actual paso, si existe una coincidencia entonces se cambia el color y el estado de los agentes de Piso Sucio.
#####Si la densidad de piso sucio llega a 0, se detiene el proceso
#####**Step**
#####Ejecuta la función de buscar una nueva celda para realizar el movimiento de los Roombas
#####También aumenta el contador de movimientos realizados

In [28]:
class CleaningModel(ap.Model):

    def setup(self):

        # Parameters
        h = self.p.height
        w = self.p.width
        d = self.d = int(self.p.density * (w * h))
        n = self.n = self.p.n_agents
        self.num_moves = 0

        # Create grid and agents
        self.grid = ap.Grid(self, (w, h), track_empty=True, check_border=True)
        self.robotAgents = ap.AgentList(self, n, Robot)
        self.cellAgents = ap.AgentList(self, d, Cell)
        self.grid.add_agents(self.cellAgents, empty=False, random=True)
        self.grid.add_agents(self.robotAgents, positions = None, empty=False, random=False)
  

    def update(self):
        # Move unhappy people to new location
        self.allRobots = self.robotAgents.select(self.robotAgents.id > 0)
        self.allCells = self.cellAgents.select(self.cellAgents.typeColor == 1)

        for robot in self.allRobots:
          for agent in self.allCells:
            pos = self.grid.positions[agent]  # Get position
            posr = self.grid.positions[robot]
            if pos == posr and agent.clean == False:
              # print(pos, posr)
              self.d-=1
              # print(self.d)
              agent.clean = True
              agent.typeColor = 2
            
        # Stop simulation if no fire is left
        if self.d == 0:
            self.stop()

    def step(self):
        # Update list of unhappy people
        self.allRobots.find_new_cell()
        self.num_moves += self.p.n_agents
        # print(self.num_moves)

####*Definimos los parámetros especificados para el ejercicio del escenario*

In [25]:
parameters = {
    'n_agents': 4, # Number of groups
    'density': 0.3, # Density of population
    'height': 5,
    'width': 10,
    'steps': 50  # Maximum number of steps
    }

####*Realizamos la gráficación y animación del modelo, así como el reporte de estado en cada momento de la iteración*

In [29]:
def animation_plot(model, ax):
    gridPosition = model.grid.attr_grid('typeColor')
    color_dict = {0:'#687068', 1:'#854e2c', 2:'#ffffff', None:'#ffffff'}
    ap.gridplot(gridPosition, ax=ax, color_dict=color_dict, convert=True)
    total = model.p.height * model.p.width
    percent = (total - model.d) * 100 / (total)
    ax.set_title(f"Cleaning Roombas model \n Time-step: {model.t}, # of Moves: {model.num_moves}, % of Clean Cells: {percent}%")

fig, ax = plt.subplots()
model = CleaningModel(parameters)
animation = ap.animate(model, fig, ax, animation_plot)
IPython.display.HTML(animation.to_jshtml())